In [1]:
import myfm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from myfm.utils.benchmark_data import MovieLens100kDataManager
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
import scipy.sparse as sps
from itertools import combinations
import numpy as np

In [2]:
path = 'E:\\recommendation-system\\CONTROL\\movies_metadata.csv'

In [3]:
df = pd.read_csv(path)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13500\3280352130.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [45]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [7]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [4]:
df = df.dropna(subset=['vote_count'])

In [5]:
df.fillna('NaN').reset_index()

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45455,45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45456,45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45457,45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45458,45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [28]:
df2 = df.copy()
df2 = df2.drop(columns=["id", "title", "original_title", "poster_path","imdb_id","spoken_languages","homepage","tagline"], axis=1)

In [29]:
df2

,adult,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,Released,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,Released,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,Released,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,Released,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,Released,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",fa,Rising and falling between a man and woman.,0.072051,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,Released,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",tl,An artist struggles to finish his work while a...,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,Released,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,Released,False,3.8,6.0
45464,False,NaN,0,[],en,"In a small town live two brothers, one a minis...",0.003503,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,Released,False,0.0,0.0


In [30]:
for column in df2.columns.values:
    print(column)
    if column not in ["budget","popularity","release_date","revenue","runtime","vote_average","vote_count"]:
        uniqe_value = df2[column].unique()
        print(uniqe_value)
        # new_value = {uniqe_value[i]: i for i in range(len(uniqe_value))} 
        df2[[column]] = df2[[column]].replace(uniqe_value,[ i for i in range(len(uniqe_value))])

adult
['False' 'True']
belongs_to_collection
["{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"
 nan
 "{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}"
 ...
 "{'id': 148603, 'name': 'Ducobu Collection', 'poster_path': '/rd7AWZUy2QFPIblNWToVmdfXQcA.jpg', 'backdrop_path': '/7mzKmoIrvGapvsSbAVlX4HtCnFj.jpg'}"
 "{'id': 152918, 'name': 'Mister Blot Collection', 'poster_path': '/44PYEwwjGts8pAob59RHd6zlkKc.jpg', 'backdrop_path': '/5uoPsNiFpUYNamSGqE8okN27VRK.jpg'}"
 "{'id': 200641, 'name': 'Red Lotus Collection', 'poster_path': '/yf9Eod9ANXyHTzDpAxz9ezgvlL4.jpg', 'backdrop_path': '/3fhHbLeO3DqdHvgHg5szs399eBb.jpg'}"]
budget
genres
["[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"
 "[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'F

In [31]:
df2

,adult,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,video,vote_average,vote_count
0,0,0,30000000,0,0,0,21.946943,0,0,1995-10-30,373554033.0,81.0,0,0,7.7,5415.0
1,0,1,65000000,1,0,1,17.015539,1,0,1995-12-15,262797249.0,104.0,0,0,6.9,2413.0
2,0,2,0,2,0,2,11.7129,2,0,1995-12-22,0.0,101.0,0,0,6.5,92.0
3,0,1,16000000,3,0,3,3.859495,3,0,1995-12-22,81452156.0,127.0,0,0,6.1,34.0
4,0,3,0,4,0,4,8.387519,4,0,1995-02-10,76578911.0,106.0,0,0,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,0,1,0,47,4,44299,0.072051,49,19,NaN,0.0,90.0,0,0,4.0,1.0
45462,0,1,0,23,57,44300,0.178241,22705,111,2011-11-17,0.0,360.0,0,0,9.0,3.0
45463,0,1,0,163,0,44301,0.903007,14623,0,2003-08-01,0.0,90.0,0,0,3.8,6.0
45464,0,1,0,42,0,44302,0.003503,22706,142,1917-10-21,0.0,87.0,0,0,0.0,0.0


In [32]:
# release_date = df2[['release_date']].copy()
df2['release_date'] = pd.to_datetime(df2['release_date'])
# release_date['date'] = release_date['release_date']
# release_date = release_date.set_index('release_date')
new_realease_day = df2.resample('9M',on='release_date').count()
new_realease_day
# release_date
# df2.loc[19720:19730,['release_date']]

,adult,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,video,vote_average,vote_count
release_date,,,,,,,,,,,,,,,,
1874-12-31,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1875-09-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1876-06-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1877-03-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1877-12-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-31,68,68,68,68,68,68,68,68,68,68,68,67,68,68,68,68
2018-12-31,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
2019-09-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
indexDate = new_realease_day[ (new_realease_day['release_date'] == 0) ].index
new_realease_day.drop(indexDate , inplace=True)
new_realease_day

,adult,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,video,vote_average,vote_count
release_date,,,,,,,,,,,,,,,,
1874-12-31,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1878-09-30,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1883-12-31,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1887-09-30,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1888-06-30,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-09-30,1205,1205,1205,1205,1205,1205,1205,1205,1205,1205,1205,1200,1205,1205,1205,1205
2017-06-30,864,864,864,864,864,864,864,864,864,864,864,854,864,864,864,864
2018-03-31,68,68,68,68,68,68,68,68,68,68,68,67,68,68,68,68


In [11]:
def new_date(x,new_realease_day):
    for i in range(new_realease_day.index.shape[0]-1):
        if new_realease_day.index[i] < x <= new_realease_day.index[i+1]:
            return new_realease_day.index[i+1]

In [34]:
for i in range(new_realease_day.index.shape[0]-1):
    df2['release_date'] = df2['release_date'].apply(lambda x: new_realease_day.index[i+1] if (new_realease_day.index[i] < x <= new_realease_day.index[i+1]) else x)
# new_realease_day.index.shape[0]
# for i in range(new_realease_day.index.shape[0]-1):
#     print(new_realease_day.index[i])
#     print(new_realease_day.index[i+1])

In [35]:
unique_value = df2['release_date'].unique()
df2['release_date'] = df2['release_date'].replace(unique_value, [ i for i in range(len(unique_value))])

In [36]:
df2

,adult,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,video,vote_average,vote_count
0,0,0,30000000,0,0,0,21.946943,0,0,0,373554033.0,81.0,0,0,7.7,5415.0
1,0,1,65000000,1,0,1,17.015539,1,0,0,262797249.0,104.0,0,0,6.9,2413.0
2,0,2,0,2,0,2,11.7129,2,0,0,0.0,101.0,0,0,6.5,92.0
3,0,1,16000000,3,0,3,3.859495,3,0,0,81452156.0,127.0,0,0,6.1,34.0
4,0,3,0,4,0,4,8.387519,4,0,1,76578911.0,106.0,0,0,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,0,1,0,47,4,44299,0.072051,49,19,33,0.0,90.0,0,0,4.0,1.0
45462,0,1,0,23,57,44300,0.178241,22705,111,106,0.0,360.0,0,0,9.0,3.0
45463,0,1,0,163,0,44301,0.903007,14623,0,120,0.0,90.0,0,0,3.8,6.0
45464,0,1,0,42,0,44302,0.003503,22706,142,113,0.0,87.0,0,0,0.0,0.0


In [37]:
df2 = df2.astype('float')
# result_corrcoef = np.corrcoef(df2.values,dtype=np.float32)
corrcoef = df2.corr()
# df3.values

In [38]:
corrcoef

,adult,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,video,vote_average,vote_count
adult,1.000000,-0.003877,-0.003345,-0.001717,0.001816,0.013846,-0.003392,0.004979,0.003044,0.003416,-0.002452,-0.009029,0.024722,-0.000637,-0.012403,-0.002922
belongs_to_collection,-0.003877,1.000000,0.095217,0.123838,0.040411,0.052375,0.106432,0.056651,-0.014303,0.010997,0.126614,-0.012513,-0.003281,-0.004234,0.032529,0.120573
budget,-0.003345,0.095217,1.000000,0.051496,-0.076111,-0.141494,0.449645,0.007455,-0.005112,0.072872,0.768776,0.134733,-0.021247,-0.010831,0.073494,0.676642
genres,-0.001717,0.123838,0.051496,1.000000,0.031193,0.119053,0.047453,0.120329,0.015335,0.042551,0.040049,0.011377,0.002802,0.012733,0.005846,0.038431
original_language,0.001816,0.040411,-0.076111,0.031193,1.000000,0.123513,-0.092035,0.137573,0.247596,0.070585,-0.056207,0.080159,0.006856,-0.007342,0.061525,-0.067054
overview,0.013846,0.052375,-0.141494,0.119053,0.123513,1.000000,-0.174536,0.435500,0.124073,0.317342,-0.104679,-0.116749,0.040672,0.032262,-0.141162,-0.109978
popularity,-0.003392,0.106432,0.449645,0.047453,-0.092035,-0.174536,1.000000,0.040691,-0.016836,0.072517,0.506179,0.129927,-0.022469,-0.015574,0.154399,0.559965
production_companies,0.004979,0.056651,0.007455,0.120329,0.137573,0.435500,0.040691,1.000000,0.214446,0.274295,0.000816,0.023021,0.009463,-0.007710,0.023757,0.024833
production_countries,0.003044,-0.014303,-0.005112,0.015335,0.247596,0.124073,-0.016836,0.214446,1.000000,0.104585,-0.020030,0.043581,0.001719,-0.010683,0.034870,-0.012546
release_date,0.003416,0.010997,0.072872,0.042551,0.070585,0.317342,0.072517,0.274295,0.104585,1.000000,0.044892,-0.038745,0.033822,0.013400,0.019636,0.065646


In [39]:
for i in corrcoef.columns.values:
    print("column ",i)
    print(corrcoef.sort_values(by=[i],ascending=False)[i])
    print("----------------------------")

column  adult
adult                    1.000000
status                   0.024722
overview                 0.013846
production_companies     0.004979
release_date             0.003416
production_countries     0.003044
original_language        0.001816
video                   -0.000637
genres                  -0.001717
revenue                 -0.002452
vote_count              -0.002922
budget                  -0.003345
popularity              -0.003392
belongs_to_collection   -0.003877
runtime                 -0.009029
vote_average            -0.012403
Name: adult, dtype: float64
----------------------------
column  belongs_to_collection
belongs_to_collection    1.000000
revenue                  0.126614
genres                   0.123838
vote_count               0.120573
popularity               0.106432
budget                   0.095217
production_companies     0.056651
overview                 0.052375
original_language        0.040411
vote_average             0.032529
release_date   